In [ ]:
!pip install -q google-genai

In [ ]:
from google import genai

In [ ]:
!gcloud auth login

In [ ]:
# Only run this block for Vertex AI API
client = genai.Client(vertexai=True, project="sandbox-455622", location="us-central1")

# Sequential Agent

In [ ]:
# --- 1. Instantiate the model(s) ---
# The global `genai` configuration (or the `client` created in a previous
# notebook cell) is already set up, so we can request models right away.
story_model = "gemini-2.0-flash-001"
critic_model = "gemini-2.0-flash-001"

# --- 2. Generate the short story ---
story_prompt = (
    "You are a creative short story writer.\n"
    "Write a brief, engaging story (3–4 paragraphs) about an unexpected "
    "adventure.\n"
    "Be imaginative but concise.\n"
    "User Input: "
)
story_response = client.models.generate_content(
    model=story_model,
    contents=story_prompt + input("Please enter a prompt or seed for your story: "),
)
generated_story = story_response.text.strip()

# --- 3. Critique the generated story ---
critic_prompt = (
    "You are a literary critic.\n"
    "Analyze the provided story for its strengths and weaknesses.\n"
    "Comment on plot, characters, and overall impact.\n"
    "Provide 2–3 specific suggestions for improvement.\n\n"
    f"Story:\n{generated_story}"
)
critic_response = client.models.generate_content(model=critic_model, contents=critic_prompt)
story_critique = critic_response.text.strip()

# --- 4. Display the results ---
print("=== Generated Story ===\n")
print(generated_story)
print("\n=== Critique ===\n")
print(story_critique)

# --- 5. Improve the story based on the critique ---
revision_prompt = (
    "You are a creative short story writer.\n"
    "Revise the following story based on the critique provided.\n\n"
    f"Critique:\n{story_critique}\n\n"
    f"Original Story:\n{generated_story}\n\n"
    "Please produce an improved version of the story, addressing the suggestions.\n"
)
revision_response = client.models.generate_content(model=story_model, contents=revision_prompt)
improved_story = revision_response.text.strip()

# --- 6. Display the improved story ---
print("\n=== Improved Story ===\n")
print(improved_story)

# Evaluator Optimizer

In [ ]:
# --- 1. Instantiate the model(s) ---
# The global `genai` configuration (or the `client` created in a previous
# notebook cell) is already set up, so we can request models right away.
story_model = "gemini-2.0-flash-001"
critic_model = "gemini-2.0-flash-001"

# --- Configuration for iterative improvement ---
num_iterations = 2  # Set the number of critique/revision cycles

# --- 2. Generate the initial short story ---
story_prompt = (
    "You are a creative short story writer.\n"
    "Write a brief, engaging story (3–4 paragraphs) about an unexpected "
    "adventure.\n"
    "Be imaginative but concise.\n"
    "User Input: "
)
story_response = client.models.generate_content(
    model=story_model,
    contents=story_prompt + input("Please enter a prompt or seed for your story: "),
)
# Initialize the story variable that will be updated in the loop
current_story = story_response.text.strip()

print("=== Initial Story ===\n")
print(current_story)

# --- 3. Iteratively Critique and Improve the story ---
for i in range(num_iterations):
    print(f"\n--- Iteration {i+1} ---")

    # --- 3a. Critique the current story ---
    print("\nGenerating critique...")
    critic_prompt = (
        "You are a literary critic.\n"
        "Analyze the provided story for its strengths and weaknesses.\n"
        "Comment on plot, characters, and overall impact.\n"
        "Provide 2–3 specific suggestions for improvement.\n\n"
        f"Story:\n{current_story}"
    )
    critic_response = client.models.generate_content(model=critic_model, contents=critic_prompt)
    story_critique = critic_response.text.strip()

    # --- 3b. Display the critique ---
    print(f"\n=== Critique {i+1} ===\n")
    print(story_critique)

    # --- 3c. Improve the story based on the critique ---
    print("\nGenerating revision...")
    revision_prompt = (
        "You are a creative short story writer.\n"
        "Revise the following story based *only* on the critique provided.\n"
        "Focus on addressing the specific suggestions mentioned in the critique.\n"
        "Do not introduce significant new elements not prompted by the critique.\n\n"
        f"Critique:\n{story_critique}\n\n"
        f"Original Story:\n{current_story}\n\n"
        "Please produce an improved version of the story, addressing the suggestions.\n"
        "Output *only* the revised story."
    )
    revision_response = client.models.generate_content(model=story_model, contents=revision_prompt)
    # Update the current story with the improved version for the next iteration
    current_story = revision_response.text.strip()

    # --- 3d. Display the improved story for this iteration ---
    print(f"\n=== Improved Story {i+1} ===\n")
    print(current_story)

print("\n--- Final Story after Iterations ---")
print(current_story)

# Parallelization

In [ ]:
import concurrent.futures
import textwrap

GEMINI_MODEL_NAME = "gemini-2.0-flash-001"

# --- 1. Define Research Prompts ---
# We define the prompts directly, without ADK agent wrappers.

research_prompts = {
    "ai_research_result": (
        "Artificial Intelligence",
        """You are an AI Research Assistant.
Research the latest advancements in 'Artificial Intelligence'.
Summarize your key findings concisely (1-2 sentences).
Focus on information readily available up to your knowledge cutoff.
Output *only* the summary.""",
    ),
    "quantum_research_result": (
        "Quantum Computing",
        """You are an AI Research Assistant specializing in physics and computing.
Research the latest breakthroughs in 'Quantum Computing'.
Summarize your key findings concisely (1-2 sentences).
Focus on information readily available up to your knowledge cutoff.
Output *only* the summary.""",
    ),
    "biotech_research_result": (
        "Biotechnology",
        """You are an AI Research Assistant specializing in life sciences.
Research the latest innovations in 'Biotechnology'.
Summarize your key findings concisely (1-2 sentences).
Focus on information readily available up to your knowledge cutoff.
Output *only* the summary.""",
    ),
}

# --- 2. Execute Research Tasks in Parallel ---


# Function to call the Gemini API for a single research task
def run_research_task(topic, prompt):
    """Calls the generative model and returns the text result."""
    print(f"Starting research for: {topic}...")
    try:
        response = client.models.generate_content(model=GEMINI_MODEL_NAME, contents=prompt)
        print(f"Finished research for: {topic}.")
        return response.text.strip()
    except Exception as e:
        print(f"Error during research for {topic}: {e}")
        return f"Error retrieving information for {topic}."


# Use ThreadPoolExecutor for parallel execution
research_results = {}
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Submit tasks
    future_to_key = {
        executor.submit(run_research_task, topic, prompt): key
        for key, (topic, prompt) in research_prompts.items()
    }
    # Collect results as they complete
    for future in concurrent.futures.as_completed(future_to_key):
        key = future_to_key[future]
        try:
            result = future.result()
            research_results[key] = result
        except Exception as exc:
            print(f"{key} generated an exception: {exc}")
            research_results[key] = f"Error in {key} task."

print("\n--- Parallel Research Results ---")
for key, result in research_results.items():
    print(f"{key}: {result}")

# --- 3. Define and Execute the Synthesis Task ---

# Prepare the synthesis prompt using the collected results
synthesis_instruction = f"""You are an AI Assistant responsible for combining research findings into a structured report.

Your primary task is to synthesize the following research summaries, clearly attributing findings to their source areas (AI, Quantum Computing, Biotechnology). Structure your response using headings for each topic. Ensure the report is coherent and integrates the key points smoothly.

**Crucially: Your entire response MUST be grounded *exclusively* on the information provided in the 'Input Summaries' below. Do NOT add any external knowledge, facts, or details not present in these specific summaries.**

**Input Summaries:**

*   **AI Advancements:** {research_results.get('ai_research_result', 'N/A')}
*   **Quantum Computing:** {research_results.get('quantum_research_result', 'N/A')}
*   **Biotechnology:** {research_results.get('biotech_research_result', 'N/A')}

Produce the final synthesized report.
"""

print("\n--- Starting Synthesis ---")
try:
    synthesis_response = client.models.generate_content(
        model=GEMINI_MODEL_NAME, contents=synthesis_instruction
    )
    final_report = synthesis_response.text.strip()
    print("--- Synthesis Complete ---")
except Exception as e:
    print(f"Error during synthesis: {e}")
    final_report = "Error generating the final report."


# --- 4. Display the Final Report ---
print("\n=== Final Synthesized Report ===\n")
# Use textwrap for potentially long reports in notebooks
print(textwrap.fill(final_report, width=80))

# Routing

In [ ]:
from google.genai import types

In [ ]:
def get_current_weather(location: str) -> str:
    """Returns the current weather.

    Args:
      location: The city and state, e.g. San Francisco, CA
    """
    return "sunny"


def get_current_time(location: str) -> str:
    """Returns the current time.

    Args:
      location: The city and state, e.g. San Francisco, CA
    """
    return "10:00 AM"


response = client.models.generate_content(
    model="gemini-2.0-flash-001",
    contents="What is the weather like in Boston?",
    config=types.GenerateContentConfig(tools=[get_current_weather, get_current_time]),
)

print(response.text)

response = client.models.generate_content(
    model="gemini-2.0-flash-001",
    contents="What is the time in Boston?",
    config=types.GenerateContentConfig(tools=[get_current_weather, get_current_time]),
)

print(response.text)